In [6]:
import numpy as np

class Embedding:
    def __init__(self, w):
        self.params = [w]
        self.grads = [np.zeros_like(w)]
        self.idx = None
    def forward(self, idx):
        self.idx = idx
        return self.params[0][idx]
    def backward(self, dout):
        self.grads[0][...] = 0
        np.add.at(self.grads[0], self.idx, dout)
        return None

In [5]:
class EmbeddingDot:
    def __init__(self, w):
        self.embed = Embedding(w)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None
    def forward(self, h, idx):
        target_w = self.embed.forward(idx)
        out = np.sum(target_w*h, axis=1)
        self.cache = (h, target)
        return out
    def backward(self, dout):
        h, target_w = self.cache
        self.embed.backward(h*dout)
        return target_w*dout

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 7,  8,  9],
       [10, 11, 12],
       [13, 14, 15],
       [15, 16, 17],
       [18, 19, 20]])